**Reducción datos archivo original.** Esto con intención de tener la primera base de datos para la visualización de ***ARTISTAS + SELLOS DISCOGRAFICOS.***

1. *En función de evidenciar el dominio de ciertas empresas en el mercado estadounidense*, **se realiza un aislamiento de datos**, teniendo como enfoque el top 10 del Hot 100, por ende considerango el rango de añs como 2000-2025, **se edita el csv completo para quedar con las canciones separando a los artistas de forma individual**. ejemplo: Golden de Huntrix ahora se desgloza asociado a las 3 vocalistas de forma separada.

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/csv/hot100_top10_only.csv")

df_artists = df[['Date', 'Song', 'Artist']].copy()

df_artists['Artist_Clean'] = (df_artists['Artist']
                              .str.replace(r'^"|"$', '', regex=True)
                              .str.replace(r'""', '"', regex=True)
                             )

df_artists['Artist_Clean'] = (df_artists['Artist_Clean']
                              .str.replace(r'\s+Featuring\s+', '|', regex=True)
                              .str.replace(r'\s+&\s+', '|', regex=True)
                              .str.replace(r'\s+with\s+', '|', regex=True)
                              .str.replace(r':\s+', '|', regex=True)
                             )

df_artists['Individual_Artist'] = df_artists['Artist_Clean'].str.split('|')
df_artists_exploded = df_artists.explode('Individual_Artist')


df_artists_exploded['Individual_Artist'] = df_artists_exploded['Individual_Artist'].str.strip()
df_artists_only = df_artists_exploded[['Date', 'Song', 'Individual_Artist']].rename(columns={'Individual_Artist': 'Artist'})
df_artists_only = df_artists_only.drop_duplicates().reset_index(drop=True)

df_artists_only.to_csv("individual_artists.csv", index=False)

2. Ahora se ejecuta el codigo para incorporar los sellos discograficos a cada artista. Con esto realizado, el archivo seria ya base para las visualizaciones.

In [ ]:
import pandas as pd
import requests
import time
import urllib.parse


df_artists_only = pd.read_csv("/content/drive/MyDrive/csv/individual_artists.csv")

df_artists_only['Record_Label'] = None


GENIUS_ACCESS_TOKEN = "e87LgpoSjehzdKaXfFTU1USopb4yCPNI047aAgqd-OQEacCxNd_ybcll3wIxvj4P"
BASE_URL = "https://api.genius.com/"
HEADERS = {
    'Authorization': f'Bearer {GENIUS_ACCESS_TOKEN}',
}

for index, row in df_artists_only.iterrows():
    artist = row['Artist']
    song = row['Song']

    search_query = f"{song} {artist}"

    try:
        search_endpoint = BASE_URL + "search"
        params = {'q': search_query}

        response = requests.get(search_endpoint, headers=HEADERS, params=params)
        response.raise_for_status()
        search_data = response.json()


        song_id = None
        if search_data.get('response', {}).get('hits'):
            best_hit = search_data['response']['hits'][0]['result']
            song_id = best_hit['id']


            if song_id:
                song_endpoint = BASE_URL + f"songs/{song_id}"

                response_song = requests.get(song_endpoint, headers=HEADERS)
                response_song.raise_for_status()
                song_data = response_song.json()


                title = song_data['response']['song']['title_with_featured']
                df_artists_only.loc[index, 'Record_Label'] = f"Datos_obtenidos ({title})"

    except requests.exceptions.RequestException as e:
        df_artists_only.loc[index, 'Record_Label'] = f"Error_API: {e}"

    time.sleep(2)

df_artists_only.to_csv("artistas_genius_status.csv", index=False)

Al final, debido a lo masivo de la cantidad de datos y el continuo error, no segui trabajando con este.

Por otro lado, a pesar de en pruebas haber llegado al resultado deseado de sellos + artistas, entre el archivo pasado por limpieza y el que se le sumaron sellos, al ultimo le faltaron varios artistas. Por lo que su uso implicaria un verdad imcopleta.